In [21]:
import json
import re

class TextoJson:
    def __init__(self, nombreArchivo):
        self.nombreArchivo = nombreArchivo
        self.informacion = None
        # vectores
        self.data = {
            "antecedentes_personales": {},
            "antecedentes_familiares": {},
            "estadia_tumoral": {},
            "inmunohistoquimica_tumoral": {},
            "tipo": {},
        }

        # Cargar datos al inicializar el objeto
        self.cargar_datos()

    def cargar_datos(self):
        with open(self.nombreArchivo, 'r', encoding='utf-8') as file:
            self.informacion = file.readlines()

    #estructura la data a json
    def estructurar_data(self):
        return json.dumps(self.data, indent=4, ensure_ascii=False)
    
    def imprimir_data(self):
        print(json.dumps(self.data, indent=4, ensure_ascii=False))

    # Función para limpiar el texto
    def limpiar_texto_resumen(self):
        # Unir la lista de líneas en un solo string
        texto = ' '.join(self.informacion)

        # Eliminar saltos de línea y unir líneas que pertenecen al mismo párrafo
        texto = re.sub(r'\n+', ' ', texto)
        
        # Eliminar espacios adicionales
        texto = re.sub(r'\s+', ' ', texto)
        
        # Separar las líneas que comienzan con fechas (formato dd.mm.aa)
        texto = re.sub(r'(\d{2}\.\d{2}\.\d{2})', r'\n\1', texto)
        
        # Separar el encabezado y el resumen
        if "RESUMEN DEL PADECIMIENTO ONCOLÓGICO" in texto:
            texto = re.sub(r'(RESUMEN DEL PADECIMIENTO ONCOLÓGICO)', r'\n\1\n', texto)
        
        # Eliminar espacios al inicio y al final
        texto = texto.strip()

        #volvemos a convertir el texto a una lista de lineas
        self.informacion = texto.split('\n')

    def preprocesar_texto(self):

        #si en self.informacion hay una linea con "RESUMEN DEL PADECIMIENTO ONCOLÓGICO" se manda a llama a la funcion limpiar_texto
        if any("RESUMEN DEL PADECIMIENTO ONCOLÓGICO" in linea for linea in self.informacion):
            self.limpiar_texto_resumen()

        texto_procesado = []
        clave_actual = None
        valor_actual = []

        for linea in self.informacion:
            linea = linea.strip().lower()

            if not linea:
                continue  # Ignorar líneas vacías

            # Identificar si la línea es una nueva clave
            if re.match(r'^[a-záéíóúñ]+.*?:', linea, re.IGNORECASE) or re.match(r'^-\s*[a-záéíóúñ]+.*?', linea, re.IGNORECASE):
                # Si ya hay una clave en curso, guardar la clave y su valor
                if clave_actual:
                    if valor_actual:  # Si tiene valores asociados
                        texto_procesado.append(f"{clave_actual}: {' '.join(valor_actual).strip()}")
                    else:  # Si no tiene valores, solo agregar la clave
                        texto_procesado.append(clave_actual)
                
                # Actualizar clave y reiniciar el valor
                if ':' in linea:
                    # Eliminar "-" inicial y espacios en caso de que existan
                    linea = re.sub(r'^-\s*', '', linea)
                    clave_actual = linea.split(':', 1)[0].strip()
                    valor_actual = [linea.split(':', 1)[1].strip()]
                else:
                    clave_actual = linea.strip('- ').strip()
                    valor_actual = []
            elif re.match(r'^[+-]?\s*cáncer\b', linea, re.IGNORECASE):
                # Si la línea contiene "cáncer", agregarla directamente al texto procesado
                texto_procesado.append(f"diagnóstico: {linea}")
                

            elif clave_actual:
                # Agregar la línea al valor actual
                valor_actual.append(linea)
            else:
                # Este caso es cuando la clave no tiene dos puntos
                texto_procesado.append(linea)

        # Guardar la última clave-valor si quedó algo pendiente
        if clave_actual:
            texto_procesado.append(f"{clave_actual}: {' '.join(valor_actual).strip()}")

        texto_procesado = [re.sub(r'^[^a-zA-Z0-9]+', '', linea) for linea in texto_procesado]

        print('\n'.join(texto_procesado))
        return '\n'.join(texto_procesado)
    
    def extraer_datos_antecedentes_personales(self):
        # secciones de antecedentes_personales
        antecedentes_personales_secciones = {
            "edad": None,
            "sexo": "femenino",
            "peso": None,
            "talla": None,
            "preferencia": None,
            "índice_tabáquico": None,
            "alcohol": None,
            "drogas": None,
            "comorbilidades": None,
            "antecedentes_ginecológicos": {
                "fum": None,
                "menarca": None,
                "embarazos": None,
                "partos": None,
                "trh": None,
                "estado_hormonal": None,
                "métodos_anticonceptivos": None
            }
        }

        #palaras clave para antecedentes_personales que buscaremos en el texto
        antecedentes_personales_keywords = [
            "edad", "sexo", "peso", "talla", "preferencia", 
            "índice tabáquico", "tabaco", "tabaquismo", "alcohol", "drogas", 
            "comorbilidades", "antecedentes ginecológicos", "menarca", "embarazos", "partos", "fum", "trh", 
            "estado hormonal", "métodos anticonceptivos"
        ]

        texto_procesado = self.preprocesar_texto()
        isEdad = False

        # Agregar una regex para detectar edades en el formato "XX años"
        edad_regex = re.compile(
            r'\b(?:edad:?\s*(\d+)(?:\s*años)?)|(?:\b(\d+)\s*años(?:\s*de\s*edad)?)|^(\w+)\s*/\s*(.+?)\s*/\s*(\d+\s*años)\s*/\s*(.*)$',            re.IGNORECASE
        )
        for linea in texto_procesado.splitlines():
            # Buscar una edad en el formato específico
            match = edad_regex.match(linea)
            if match:
                if match.group(1):  # "edad: XX" o "edad: XX años"
                    antecedentes_personales_secciones["edad"] = match.group(1).strip()
                elif match.group(2):  # "XX años" o "XX años de edad"
                    antecedentes_personales_secciones["edad"] = match.group(2).strip()
                elif match.group(5):  # Formato "gacmc / ... / XX años / ..."
                    antecedentes_personales_secciones["edad"] = match.group(5).strip()
                isEdad = True
            
            for keyword in antecedentes_personales_keywords:
                if keyword in linea:
                    #si encuentra la palabra "edad"
                    if keyword == "edad" and not isEdad:
                        if ":" in linea:
                            antecedentes_personales_secciones["edad"] = linea.split(":")[1].strip()
                        else:
                            antecedentes_personales_secciones["edad"] = linea.split(keyword)[1].strip()
                        isEdad = True
                    
                    #si la keyword es "tabaquismo" o "tabaco" o "índice tabáquico" y si tiene o no ":" que separan_
                    elif keyword == "tabaquismo" or keyword == "tabaco" or keyword == "índice tabáquico":
                        if ":" in linea:
                            antecedentes_personales_secciones["índice_tabáquico"] = linea.split(":")[1].strip()
                        else:
                            antecedentes_personales_secciones["índice_tabáquico"] = linea.split(keyword)[1].strip()

                    elif keyword in ["fum", "menarca", "trh", "estado hormonal", "métodos anticonceptivos", "embarazos", "partos"]:
                        # Verificar las keywords de antecedentes ginecológicos en la misma línea
                        pattern = re.compile(r'(fum|menarca|trh|estado hormonal|métodos anticonceptivos|embarazos|partos)', re.IGNORECASE)
                        matches = pattern.findall(linea)
                        if len(matches) > 1:
                           # Dividir la línea en partes basadas en varios separadores posibles
                            partes = re.split(r'[\/,\.]', linea)
                            for parte in partes:
                                # Asegurarte de que cada elemento sea una cadena válida
                                if isinstance(parte, str):  # Por seguridad, verifica si el elemento es una cadena
                                    parte = parte.strip()  # Elimina espacios al inicio y al final
                                    
                                for keyword in matches:
                                    if keyword in parte:

                                        if ":" in parte:
                                            antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().lower()] = parte.split(":")[1].strip()
                                            #self.asignar_si_vacio(antecedentes_personales_secciones["antecedentes_ginecológicos"], keyword.strip().lower(), parte.split(":")[1].strip())
                                        else:
                                            # Verificar si el valor precede a la clave
                                            if parte.strip().startswith(keyword):
                                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().lower()] = parte.split(keyword)[1].strip()
                                                #self.asignar_si_vacio(antecedentes_personales_secciones["antecedentes_ginecológicos"], keyword.strip().lower(), parte.split(match)[1].strip())
                                            else:
                                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().lower()] = parte.split(keyword)[0].strip()
                                                #self.asignar_si_vacio(antecedentes_personales_secciones["antecedentes_ginecológicos"], keyword.strip().lower(), parte.split(keyword)[0].strip())
                            continue
                        
                        else:
                            if ":" in linea:
                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.replace(" ", "_")] = linea.split(":")[1].strip()
                            else:
                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.replace(" ", "_")] = linea.split(keyword)[1].strip()
                    
                    ##tratamos las comoorbilidades dado que es una linea con varias comorbilidades separadas por "/"
                    elif keyword == "comorbilidades":
                        comorbilidades = linea
                        # Dividir la cadena en comorbilidades individuales
                        comorbilidades_list = re.split(r'\d+\.\s', comorbilidades)

                        # Eliminar posibles cadenas vacías resultantes de la división
                        comorbilidades_list = [comorbilidad for comorbilidad in comorbilidades_list if comorbilidad]

                        comorbilidades_dict = {}

                        for comorbilidad in comorbilidades_list:
                            comorbilidad = comorbilidad.strip()
                            # Extraer el año
                            match = re.search(r'\((\d{4})\)', comorbilidad)
                            if match:
                                year = match.group(1)
                                # Separar nombre y descripción
                                parts = comorbilidad.split(f"({year})")
                                nombre = parts[0].strip()
                                descripcion = parts[1].strip() if len(parts) > 1 else ""

                                # Si la descripción es solo un punto, lo agregamos al nombre
                                if descripcion == ".":
                                    descripcion = None  # Dejamos la descripción vacía
                                    nombre = nombre.rstrip('.')  # Quitamos el punto del nombre

                                if year not in comorbilidades_dict:
                                    comorbilidades_dict[year] = []

                                comorbilidades_dict[year].append({
                                    "nombre": nombre.replace("comorbilidades: ", "").strip(),
                                    "descripción": descripcion.lstrip(': ') if descripcion else None
                                })
                        antecedentes_personales_secciones["comorbilidades"] = comorbilidades_dict

                    else:
                        if keyword == "edad" and isEdad == True:
                            continue
                        elif ":" in linea:
                            antecedentes_personales_secciones[keyword.replace(" ", "_")] = linea.split(":")[1].strip()
                        
                        else:
                            antecedentes_personales_secciones[keyword.replace(" ", "_")] = linea.split(keyword)[1].strip()

        self.data["antecedentes_personales"] = antecedentes_personales_secciones
        #self.estructurar_data()     
        self.imprimir_data()
    

procesador = TextoJson('D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\ExtraccionTexto\TextoExtraido3.txt')
procesador.extraer_datos_antecedentes_personales()

gacmc / 84959 / 67 años / dra. martínez
originaria y residente: guanajuato y ciudad de méxico
ocupación: directora de planeación conalep
seguridad social: issste
diagnóstico: cdi de mama derecha ec !a triple negativo
ahf oncológicos: tía materna falleció por cáncer de mama / prima materna con cáncer de mama (desconoce edad al diagnóstico)
tabaco: negado
alcohol: ocasional
drogas: negado
menarca 15 años / fum 21 años
cirugías: esplenectomía abierta (1976) / tiroidectomía (1988) / pancreatectomia distal + resección de implante gástrico + biopsia en cuña de segmento ii hepático + colecistectomía y maniobra de kocher extendida (2016) / cc + gc (2017)
g0 p0 c0 a0
trh: durante 30 años
estado hormonal: postmenopáusica
comorbilidades: 1. linfoma de hodakin (1975): 3 ciclos qt (ciclofosfamida, vincristina y otro fármaco) + rt por encima del diafragma (imss) 2. cáncer papilar de tiroides (1988): tiroidectomía + iodo 131 t4/t3a 3. cistoadenoma seroso microquístico de páncreas (2016) 4. cdi de mam

<>:264: SyntaxWarning: invalid escape sequence '\D'
<>:264: SyntaxWarning: invalid escape sequence '\D'
C:\Users\JOSUE\AppData\Local\Temp\ipykernel_6876\2284392036.py:264: SyntaxWarning: invalid escape sequence '\D'
  procesador = TextoJson('D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\ExtraccionTexto\TextoExtraido3.txt')


In [30]:
import re

class DataExtractor:
    def __init__(self, data):
        self.data = data

    def extract_units(self):
        # Regular expression to match patterns like "años", "meses", "días"
        unit_pattern = re.compile(r'\b(\d+)\s*(años|meses|días|horas)\b|\b(años|meses|días|horas)\s*(\d+)\b')
        matches = unit_pattern.findall(self.data)
        
        units = []
        for match in matches:
            if match[0] and match[1]:
                units.append((match[0], match[1]))
            elif match[2] and match[3]:
                units.append((match[3], match[2]))
        
        return units

# Ejemplo de uso
data = "Menarca a los 14 años"
extractor = DataExtractor(data)
unidades = extractor.extract_units()
for unidad, tipo_unidad in unidades:
    print(f"Unidad: {unidad}, Tipo de unidad: {tipo_unidad}")

Unidad: 14, Tipo de unidad: años
